# 第3章 タイトル未定

本章では文書分類モデルの作成を通じて自然言語処理で用いられる数々のPythonツールの利用方法を学びます。

なお、動作確認は以下の環境で行いました。

- Machine (AWS EC2 p2.xlargeインスタンス)
    - OS: Ubuntu 16.04
    - CPU: Intel(R) Xeon(R) CPU E5-2686 v4 @ 2.30GHz
    - RAM: 64GB 
    - GPU: NVIDIA Tesla K80
- Python
    - python: 3.7.5
    - mecab-python3: 0.996.2
    - torch (PyTorch): 1.3.1
    - torchtext: 0.4.0
    - transformers 2.3.0
    - spaCy 2.2.3
    - cupy 7.0.0
    
## 3.1 機械学習モデル開発のワークフローと本章で扱う内容

文書分類のモデルは基本的に**教師あり学習**の枠組みで訓練します。つまり、図のようにテキストとラベルのペアで訓練データを用意します。例えばニュース記事があるとして、その記事はスポーツニュースなのか、芸能ニュースなのか、政治ニュースなのかを記事内の文章から分類したいとします。このときニュース記事の文章と、分類すべきカテゴリーの名前をペアとして扱います。

![training.svg](../figures/training.svg)
<center>図出典: https://spacy.io/usage/training</center>

1. 訓練用のデータセットを用意する
2. 分類に用いる機械学習モデルを準備する
3. モデルに訓練データのテキストを入力して予測値を得る
4. モデルの予測と真のラベルを比較する
5. 誤差を減らすようなモデルのパラメーター (重み) の更新値 (gradient) を計算する
6. モデルのパラメーターを更新する
7. 3から6を繰り返す

## 3.2 文書分類ハンズオン

### 3.2.1 本章で扱う自然言語処理ツールの解説

#### 3.2.1.1 spaCy

spaCy (https://spacy.io/) とはExplosion AIにより開発されている自然言語処理用のライブラリです。spaCyには事前学習済みの統計モデルと単語ベクトルが付属しており、50以上の言語の形態素解析（トークン化）がサポートされています。また、品詞タグ付け、依存関係解析、固有表現抽出、およびテキスト分類のための単語バッグや簡単な畳み込みニューラルネットワークモデルも備えています。MITライセンスの下でリリースされた商用のオープンソースソフトウェアです。

spaCyのドキュメンテーション (https://spacy.io/usage/) に従ってインストールしてみましょう。環境によってインストールコマンドが異なるので適宜ドキュメンテーションを参考にしてください。ここではGPU付きのオプションでインストールします。

In [ ]:
!pip install -U spacy[cuda]
# pip install -U spacy  # GPUを利用しない場合

spaCyと同時に、Explosion AIにより開発されているThinc (https://github.com/explosion/thinc) という機械学習ライブラリや、Preferred NetworksのCUDA対応のNumPy互換行列計算ライブラリであるCuPy (https://cupy.chainer.org/) などがインストールされます。

日本語の形態素解析ツールのMeCab (http://taku910.github.io/mecab/) もインストールしましょう。

In [ ]:
!pip install mecab-python3

分かち書きのテストをしてみましょう。

In [ ]:
import spacy
nlp = spacy.blank('ja')
for word in nlp('すもももももももものうち'):
    print(word)

うまく単語ごとに分割してくれていますね。

#### 3.2.1.2 Transformers

Transformersはtransformerベースの汎用アーキテクチャ （BERT、GPT-2、RoBERTa、XLM、DistilBert、XLNet、CTRL、...） を利用するためのオープンソースのシンプルなAPIを提供しています。開発はHugging Faceにより行われており、100以上の言語に対応した事前学習済みモデルが公開されています。ディープラーニングフレームワークとしてはGoogleのTensorFlow 2.0およびFaceBookが開発しているPyTorchに対応しています。ここではPyTorchを用いることにします。

Transformersで用いることのできるモデルのリストはHugging Faceのホームページ (https://huggingface.co/models) にて公開されています。日本語用BERTモデルとしては、執筆時点 (2019年12月) では東北大学 乾・鈴木研究室が公開している以下の4つのモデルを利用可能です。

-  `bert-base-japanese`:
-  `bert-base-japanese-whole-word-masking`
-  `bert-base-japanese-char`
-  `bert-base-japanese-char-whole-word-masking`

BERTには大きく分けて `BERT-Base` (12-layer, 768-hidden, 12-heads, 110M parameters) と `BERT-Large` (24-layer, 1024-hidden, 16-heads, 340M parameters) のふたつのアーキテクチャーがあります。上記のモデルは `BERT-Base` を日本語のWikipediaのデータを用いて訓練したものです。`bert-base-japanese` はMeCabとWordPieceと呼ばれる手法を用いてテキストの分かち書きを行った後に訓練されたものであり、`bert-base-japanese-char` ではテキストを文字ごとに分割しています。

BERTの訓練時に行うタスクのひとつに、文章内のトークンをマスクし、そのトークンを予測する、というものがあります。例えば `"He likes playing the piano."` という文章を `"He likes [MASK] ##ing the piano."` という文章に変換し、`[MASK]` に含まれるトークンを予測します。なお、`##` から始まるトークンは接尾辞を表しています。このとき、単語に対応するトークンはまとめてマスクするのが Whole Word Masking と呼ばれる手法です。先ほどの文章を `"He likes [MASK] [MASK] the piano."` のようにマスクして訓練したモデルが `bert-base-japanese-whole-word-masking` および、`bert-base-japanese-char-whole-word-masking` です。

さて、transformersをインストールしてみましょう。環境によってインストールコマンドが異なります。https://pytorch.org/get-started/ を参照してください。

In [ ]:
# 2019年12月現在、NVIDIAのGPUを搭載したLinuxマシンにAnacondaでPyTorchをインストールするコマンドは以下の通りです。
!conda install pytorch torchvision cudatoolkit=10.1 -c pytorch
# LinuxでかつGPUがない場合は conda install pytorch torchvision cpuonly -c pytorch

PyTorchで自然言語処理を行うときに便利なライブラリであるtorchtextもインストールしましょう。

In [ ]:
!conda install torchtext -c pytorch

transformersのインストールもpipを用いて簡単に行えます。

In [ ]:
!pip install transformers

また、以降で補助的に利用するライブラリもインストールしてください。

In [ ]:
!pip install pandas scikit-learn seaborn mojimoji

- pandas
- scikit-learn
- seaborn
- mojimoji

transformersの `BertJapaneseTokenizer` を用いた日本語文章の分かち書きのテストもしてみましょう。

In [ ]:
from transformers import BertJapaneseTokenizer
tokenizer = BertJapaneseTokenizer.from_pretrained('bert-base-japanese-whole-word-masking')
tokenizer.tokenize('いつもプレゼンテーションの撮影に無音カメラアプリをご利用いただきありがとうございます。')

### 3.2.2 データセットの準備
#### 3.2.2.1 livedoor ニュースコーパス

今回は日本語における自然言語処理の試験用データセットとしてしばしば用いられる「livedoor ニュースコーパス」を用います。

livedoorニュースはもともと株式会社ライブドアが運営するニュースサイトでしたが、株式会社ライブドアが旧ハンゲームジャパン株式会社であるNHN Japan株式会社に買収され、現在はNHN Japanが社名変更したLINE株式会社により運営されています。livedoorニュースの記事の一部には「クリエイティブ・コモンズライセンス『表示 – 改変禁止』」が適用されており、営利目的を含めて再配布可能となっています。該当するニュース記事を2012年9月上旬に株式会社ロンウイットが収集し、HTMLタグの除去などクリーニングを施した状態で公開しているのが「livedoor ニュースコーパス」です。

livedoor ニュースコーパスは以下のリンクよりダウンロード可能です。

https://www.rondhuit.com/download.html#ldcc

オープンソースの全文検索システムApache Solrで扱いやすいようXML形式でニュースが格納されている `livedoor-news-data.tar.gz` と、シンプルに各々のニュースをテキストファイルとして扱っている `ldcc-20140209.tar.gz` が公開されています。

今回は後者の `ldcc-20140209.tar.gz` をダウンロードしてください。`tar xzvf ldcc-20140209.tar.gz` などにより解凍すると `text` という名前のディレクトリが出てきます。以下のPythonスクリプトを実行するとコーパスのダウンロードと圧縮ファイルの解凍が行われ、カレントディレクトリに `text` ディレクトリが作成されます。

In [ ]:
import os
import urllib.request
import tarfile

# dataディレクトリの作成
#os.makedirs('data', exist_ok=True)

url = 'https://www.rondhuit.com/download/ldcc-20140209.tar.gz'
file_name = 'ldcc-20140209.tar.gz'

# dataディレクトリへのlivedoor ニュースコーパスのダウンロードと解凍
if not os.path.exists(file_name):
    urllib.request.urlretrieve(url, file_name)
    # tar.gzファイルを読み込み
    with tarfile.open(file_name) as tar:
        tar.extractall()
    # tar.gzファイルを消去
    os.remove(file_name)

`text` ディレクトリの中身の構造は以下の通りです。

```
text
├── CHANGES.txt
├── README.txt
├── dokujo-tsushin
├── it-life-hack
├── kaden-channel
├── livedoor-homme
├── movie-enter
├── peachy
├── smax
├── sports-watch
└── topic-news
```

`dokujo-tsushin` から `topic-news` はディレクトリであり、それぞれにニュース記事のテキストが格納されています。

```
text
├── CHANGES.txt
├── README.txt
├── dokujo-tsushin
│   ├── LICENSE.txt
│   ├── dokujo-tsushin-4778030.txt
│   ├── dokujo-tsushin-4778031.txt
│   ├── dokujo-tsushin-4782522.txt
...（以下略）
```

ニュース提供元は以下の9つです。記事の本文だけを見て、その記事がどのカテゴリに属しているのか（独女通信のニュースなのか、ITライフハックのニュースなのか、など）を判別する文書分類モデルを作成するのが本章の目的です。

- 独女通信 (http://news.livedoor.com/category/vender/90/)
- ITライフハック (http://news.livedoor.com/category/vender/223/)
- 家電チャンネル (http://news.livedoor.com/category/vender/kadench/)
- livedoor HOMME (http://news.livedoor.com/category/vender/homme/)
- MOVIE ENTER (http://news.livedoor.com/category/vender/movie_enter/)
- Peachy (http://news.livedoor.com/category/vender/ldgirls/)
- エスマックス (http://news.livedoor.com/category/vender/smax/)
- Sports Watch (http://news.livedoor.com/category/vender/208/)
- トピックニュース (http://news.livedoor.com/category/vender/news/)

ちなみに、上記サービスのうちいくつかはドメインが変わっていたり終了しているので一部リンクが切れています。それぞれの記事ファイル（dokujo-tsushin-4778030.txtなど）は以下のフォーマットで構成されています。

- １行目: 記事のURL
- ２行目: 記事の日付
- ３行目: 記事のタイトル
- ４行目以降： 記事の本文

このままでは少し扱いづらいのでひとつのtsv (tab-separated values) にまとめます。

In [ ]:
services = [
    'dokujo-tsushin',
    'it-life-hack',
    'kaden-channel',
    'livedoor-homme',
    'movie-enter',
    'peachy',
    'smax',
    'sports-watch',
    'topic-news'
]
index = ['url', 'datetime', 'title', 'body']

In [ ]:
import os
import glob

import pandas as pd

# あまりに短い文章は除く
minimum_sentence_length = 32

# 空のPandasのDataFrameを準備
df = pd.DataFrame()

# 各サービスのディレクトリでループ
for service in services:
    print('===== processing {} ====='.format(service))
    # ニュース記事をすべて指定
    # パスの例は './text/dokujo-tsushin/dokujo-tsushin-4778030.txt'
    # LICENSE.txt は除外
    wild_card = os.path.join('text', service, service + '*.txt')
    file_paths = glob.glob(wild_card)
    # 各ニュース記事のファイルパスでループ
    for file_path in file_paths:
        # ファイルを開いて一行ずつ読み込む
        with open(file_path, 'r') as f:
            lines = f.readlines()
            # tsv のカラムを辞書型で用意
            series_dict = {'service': service}
            for num, line in enumerate(lines):
                #line = line.replace('\n', '')  # 改行を削除
                # 0, 1, 2行目はそれぞれURL, 日付, 記事タイトルに相当
                if num < len(index):
                    series_dict[index[num]] = line
                # 3行目以降は本文
                elif line != '\n' and line != '':
                    series_dict['body'] += line
                # lineが空（段落の境目もしくはファイルの末尾）の場合
                else:
                    if '関連記事' in series_dict['body']:
                        pass
                    elif '関連リンク' in series_dict['body']:
                        pass
                    # PandasのSeriesを作成し、DataFrameに追加していく
                    elif len(series_dict['body']) > minimum_sentence_length:
                        s = pd.Series(series_dict)
                        df = df.append(s, ignore_index=True)
                    # bodyを初期化
                    series_dict['body'] = ''
print('done')         

作成した `DataFrame` の最初の5行と最後の5行だけ抜き出して表示してみましょう。
それぞれの行がひとつのニュース記事に対応していることより、0行から7366行の計7367個のニュース記事があることがわかります。

In [ ]:
pd.concat([df.head(3), df.tail(3)])

In [ ]:
df['service'].value_counts()

In [ ]:
(df['service'].value_counts()).plot(kind='bar', rot=45, ylim=(0, 10000))

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df['service'] = le.fit_transform(df.service.values)
pd.concat([df.head(3), df.tail(3)])

In [ ]:
from sklearn.model_selection import train_test_split
df = df[['body', 'service']]
train_df, val_test_df = train_test_split(df, test_size=0.2)
val_df, test_df = train_test_split(val_test_df, test_size=0.5)

`DataFrame` をCSV (Comma-Separated Value) やTSV (Tab-Separated Value) で保存するには `pandas.DataFrame.to_csv` メソッドを呼び出します。ひとつ目の引数 `path_or_buf` には保存先のファイルパス（もしくはファイルオブジェクト）を指定し、ふたつ目の引数 `sep` には列のセパレーターを指定します。デフォルトでは `sep=','` となっており、セパレーターはカンマ、つまり `DataFrame` はCSVで保存されます。自然言語処理を行う場合、データ内にカンマが含まれていることがあるのでしばしばセパレーターとしてはタブ (`\t`) が用いられます。ここでは `DataFrame` をTSVの形式で保存します。

In [ ]:
train_df.to_csv('train.tsv', sep='\t', index=False)
val_df.to_csv('val.tsv', sep='\t', index=False)
test_df.to_csv('test.tsv', sep='\t', index=False)

#### ラベリング

### 前処理



#### 形態素解析
#### ストップワード除去

### 3.2.3 文書分類モデル





#### 単語バッグ  (bag-of-words)

In [1]:
import spacy
spacy.prefer_gpu()
nlp = spacy.blank('ja')
nlp.tokenizer

In [2]:
#!pip install "https://github.com/megagonlabs/ginza/releases/download/latest/ginza-latest.tar.gz"

In [3]:
#import spacy
#nlp = spacy.load('ja_ginza')

In [4]:
import pandas as pd
df_train = pd.read_csv('train.tsv', delimiter='\t')
df_val = pd.read_csv('val.tsv', delimiter='\t')
df_test = pd.read_csv('test.tsv', delimiter='\t')

In [5]:
train_texts = df_train.body
val_texts = df_val.body
test_texts = df_test.body

私の環境ではそれぞれ30秒、4秒、3秒ほどかかりました。

In [6]:
train_docs = list(nlp.pipe(train_texts))
val_docs = list(nlp.pipe(val_texts))
test_docs = list(nlp.pipe(test_texts))

In [7]:
services = [
    'dokujo-tsushin',
    'it-life-hack',
    'kaden-channel',
    'livedoor-homme',
    'movie-enter',
    'peachy',
    'smax',
    'sports-watch',
    'topic-news'
]

In [8]:
df_train.service

0        3
1        6
2        0
3        0
4        1
        ..
45299    7
45300    1
45301    4
45302    0
45303    6
Name: service, Length: 45304, dtype: int64

奇妙だけど'cats'キーが必要なのだ。
nlp.update without required annotation types. Expected top-level keys: ('words', 'tags', 'heads', 'deps', 'entities', 'cats', 'links').

In [9]:
train_cats = [{'cats': {service: service == services[idx] for service in services}}
              for idx in df_train.service]
val_cats = [{'cats': {service: service == services[idx] for service in services}}
            for idx in df_val.service]
test_cats = [{'cats': {service: service == services[idx] for service in services}}
             for idx in df_test.service]

In [10]:
train_cats[0:2]
val_cats[0:2]

[{'cats': {'dokujo-tsushin': True,
   'it-life-hack': False,
   'kaden-channel': False,
   'livedoor-homme': False,
   'movie-enter': False,
   'peachy': False,
   'smax': False,
   'sports-watch': False,
   'topic-news': False}},
 {'cats': {'dokujo-tsushin': False,
   'it-life-hack': True,
   'kaden-channel': False,
   'livedoor-homme': False,
   'movie-enter': False,
   'peachy': False,
   'smax': False,
   'sports-watch': False,
   'topic-news': False}}]

In [11]:
train_data = list(zip(train_docs, train_cats))
val_data = list(zip(val_docs, val_cats))
test_data = list(zip(test_docs, test_cats))

In [12]:
print(train_data[0][0][:40])
print(train_data[0][1])

＜症状＞　顧客の話の文脈がつかめない、的外れな質問ばかりしている気がする。 ＜効果＞　どこに着目して情報収集すればいいかが
{'cats': {'dokujo-tsushin': False, 'it-life-hack': False, 'kaden-channel': False, 'livedoor-homme': True, 'movie-enter': False, 'peachy': False, 'smax': False, 'sports-watch': False, 'topic-news': False}}


In [13]:
if "textcat" not in nlp.pipe_names:
    textcat = nlp.create_pipe("textcat", 
                              config={"exclusive_classes": True, "architecture": "bow"})
    nlp.add_pipe(textcat, last=True)

In [14]:
for service in services:
    textcat.add_label(service)
    print("Add label %s." % (service))

Add label dokujo-tsushin.
Add label it-life-hack.
Add label kaden-channel.
Add label livedoor-homme.
Add label movie-enter.
Add label peachy.
Add label smax.
Add label sports-watch.
Add label topic-news.


In [15]:
#from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import classification_report

def evaluate(tokenizer, textcat, docs, cats, verbose=False):
    y_true = [max(cat['cats'].items(), key=lambda x:x[1])[0] for cat in cats]
    #y_true = [[cat['cats'][service] for service in services] for cat in cats]
    y_pred = []
    for i, doc in enumerate(textcat.pipe(docs)):
        prediction = max(doc.cats.items(), key=lambda x:x[1])[0]  # 予測のサービス名
        #prediction = services.index(prediction)
        #one_hot_prediction = [False for _ in services]
        #one_hot_prediction[services.index(prediction)] = True
        #y_pred.append(one_hot_prediction)
        y_pred.append(prediction)
    #if verbose == False:
    #    p, r, f1 = precision_recall_fscore_support(y_true, y_pred, average="micro")[:3]    
    #    #p, r, f1 = precision_recall_fscore_support(y_true, y_pred)[:3]    
    #    return {"textcat_p": p, "textcat_r": r, "textcat_f": f1}
    #else:
    return classification_report(y_true, y_pred)

In [17]:
import random
from tqdm.notebook import tqdm
from spacy.util import minibatch, compounding

other_pipes = [pipe for pipe in nlp.pipe_names if pipe != "textcat"]
n_iter = 20

with nlp.disable_pipes(*other_pipes):  # only train textcat
    textcat = nlp.pipeline[-1][-1]
    optimizer = textcat.begin_training() # NOTE
    print("Training the model...")
    batch_sizes = compounding(4.0, 32.0, 1.001)
    num_samples = len(train_data)
    for i in range(n_iter):
        print('===== iteration {}/{} ====='.format(i, n_iter))
        losses = {}
        # batch up the examples using spaCy's minibatch
        random.shuffle(train_data)
        batches = minibatch(train_data, size=batch_sizes)  # generator
        processed = 0
        for i, batch in tqdm(enumerate(batches)):
            texts, annotations = zip(*batch)
            nlp.update(texts, annotations, sgd=optimizer, drop=0.2, losses=losses)
            processed += len(batch)
            percentage = processed / num_samples * 100.0
            #if i % 20 == 0:
            #  print("  %5.2f %% of epoch done. batch size = %d" % (percentage, len(batch)))
        with textcat.model.use_params(optimizer.averages):
            # evaluate on the dev data split off in load_data()
            #scores = evaluate(nlp.tokenizer, textcat, val_docs, val_cats, verbose=True)
            report = evaluate(nlp.tokenizer, textcat, val_docs, val_cats, verbose=True)
        #print("{:^5}\t{:^5}\t{:^5}\t{:^5}".format("LOSS", "P", "R", "F"))
            print(report)
        #print("{:^5}\t{:^5}\t{:^5}\t{:^5}".format("LOSS", "P", "R", "F"))
        #print(
        #    "{0:.3f}\t{1:.3f}\t{2:.3f}\t{3:.3f}".format(  # print a simple table
        #        losses["textcat"],
        #        scores["textcat_p"],
        #        scores["textcat_r"],
        #        scores["textcat_f"],
        #    )
        #)

Training the model...
===== iteration 0/20 =====



                precision    recall  f1-score   support

dokujo-tsushin       0.78      0.84      0.81       790
  it-life-hack       0.77      0.81      0.79       816
 kaden-channel       0.70      0.56      0.62       386
livedoor-homme       0.73      0.63      0.68       545
   movie-enter       0.82      0.87      0.85       629
        peachy       0.75      0.79      0.77       800
          smax       0.88      0.88      0.88       961
  sports-watch       0.90      0.85      0.87       366
    topic-news       0.82      0.80      0.81       370

      accuracy                           0.80      5663
     macro avg       0.79      0.78      0.79      5663
  weighted avg       0.80      0.80      0.80      5663

===== iteration 1/20 =====



                precision    recall  f1-score   support

dokujo-tsushin       0.80      0.86      0.82       790
  it-life-hack       0.80      0.82      0.81       816
 kaden-channel       0.72      0.61      0.66       386
livedoor-homme       0.76      0.66      0.71       545
   movie-enter       0.84      0.88      0.86       629
        peachy       0.76      0.79      0.78       800
          smax       0.88      0.89      0.89       961
  sports-watch       0.90      0.86      0.88       366
    topic-news       0.84      0.82      0.83       370

      accuracy                           0.81      5663
     macro avg       0.81      0.80      0.80      5663
  weighted avg       0.81      0.81      0.81      5663

===== iteration 2/20 =====



                precision    recall  f1-score   support

dokujo-tsushin       0.79      0.85      0.82       790
  it-life-hack       0.82      0.82      0.82       816
 kaden-channel       0.74      0.64      0.69       386
livedoor-homme       0.76      0.68      0.71       545
   movie-enter       0.85      0.89      0.87       629
        peachy       0.79      0.79      0.79       800
          smax       0.88      0.90      0.89       961
  sports-watch       0.88      0.88      0.88       366
    topic-news       0.85      0.82      0.83       370

      accuracy                           0.82      5663
     macro avg       0.82      0.81      0.81      5663
  weighted avg       0.82      0.82      0.82      5663

===== iteration 3/20 =====



                precision    recall  f1-score   support

dokujo-tsushin       0.80      0.84      0.82       790
  it-life-hack       0.81      0.81      0.81       816
 kaden-channel       0.73      0.65      0.69       386
livedoor-homme       0.76      0.67      0.71       545
   movie-enter       0.85      0.88      0.86       629
        peachy       0.77      0.79      0.78       800
          smax       0.87      0.90      0.89       961
  sports-watch       0.89      0.87      0.88       366
    topic-news       0.84      0.81      0.83       370

      accuracy                           0.82      5663
     macro avg       0.81      0.80      0.81      5663
  weighted avg       0.81      0.82      0.81      5663

===== iteration 4/20 =====


...


                precision    recall  f1-score   support

dokujo-tsushin       0.80      0.85      0.83       790
  it-life-hack       0.81      0.82      0.81       816
 kaden-channel       0.75      0.65      0.70       386
livedoor-homme       0.76      0.69      0.72       545
   movie-enter       0.86      0.88      0.87       629
        peachy       0.79      0.80      0.80       800
          smax       0.88      0.91      0.89       961
  sports-watch       0.88      0.87      0.87       366
    topic-news       0.84      0.81      0.82       370

      accuracy                           0.82      5663
     macro avg       0.82      0.81      0.81      5663
  weighted avg       0.82      0.82      0.82      5663

===== iteration 6/20 =====



                precision    recall  f1-score   support

dokujo-tsushin       0.80      0.85      0.82       790
  it-life-hack       0.81      0.82      0.81       816
 kaden-channel       0.76      0.66      0.71       386
livedoor-homme       0.75      0.69      0.72       545
   movie-enter       0.87      0.89      0.88       629
        peachy       0.79      0.80      0.80       800
          smax       0.88      0.91      0.90       961
  sports-watch       0.89      0.86      0.88       366
    topic-news       0.84      0.82      0.83       370

      accuracy                           0.82      5663
     macro avg       0.82      0.81      0.82      5663
  weighted avg       0.82      0.82      0.82      5663

===== iteration 7/20 =====



                precision    recall  f1-score   support

dokujo-tsushin       0.80      0.85      0.82       790
  it-life-hack       0.82      0.81      0.82       816
 kaden-channel       0.73      0.66      0.69       386
livedoor-homme       0.75      0.69      0.71       545
   movie-enter       0.87      0.90      0.88       629
        peachy       0.80      0.81      0.80       800
          smax       0.88      0.91      0.90       961
  sports-watch       0.89      0.86      0.87       366
    topic-news       0.84      0.81      0.82       370

      accuracy                           0.82      5663
     macro avg       0.82      0.81      0.81      5663
  weighted avg       0.82      0.82      0.82      5663

===== iteration 8/20 =====



                precision    recall  f1-score   support

dokujo-tsushin       0.80      0.85      0.83       790
  it-life-hack       0.82      0.82      0.82       816
 kaden-channel       0.74      0.67      0.70       386
livedoor-homme       0.75      0.68      0.71       545
   movie-enter       0.87      0.89      0.88       629
        peachy       0.79      0.80      0.80       800
          smax       0.89      0.91      0.90       961
  sports-watch       0.88      0.86      0.87       366
    topic-news       0.83      0.81      0.82       370

      accuracy                           0.82      5663
     macro avg       0.82      0.81      0.81      5663
  weighted avg       0.82      0.82      0.82      5663

===== iteration 9/20 =====


KeyboardInterrupt: 

#### 畳み込みニューラルネットワーク

In [20]:
nlp.remove_pipe('textcat')

ValueError: [E001] No component 'textcat' found in pipeline. Available names: []

In [21]:
if "textcat" not in nlp.pipe_names:
    textcat = nlp.create_pipe("textcat", config={"exclusive_classes": True, "architecture": "simple_cnn"})
    nlp.add_pipe(textcat, last=True)

for label in services:
    textcat.add_label(label)
    print("Add label %s." % (label))

Add label dokujo-tsushin.
Add label it-life-hack.
Add label kaden-channel.
Add label livedoor-homme.
Add label movie-enter.
Add label peachy.
Add label smax.
Add label sports-watch.
Add label topic-news.


In [49]:
!pip install chakin gensim

  Using cached https://files.pythonhosted.org/packages/ca/3f/ca2f63451c0ab47970a6ab1d39d96118e70b6e73125529cea767c31368a3/chakin-0.0.8-py3-none-any.whl
     |████████████████████████████████| 24.2MB 3.2MB/s eta 0:00:01
  Using cached https://files.pythonhosted.org/packages/16/68/adc395e0a3c86571081c8a2e2daaa5b58270f6854276a089a0e9b5fa2c33/progressbar2-3.47.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/0c/09/735f2786dfac9bbf39d244ce75c0313d27d4962e71e0774750dc809f2395/smart_open-1.9.0.tar.gz
  Using cached https://files.pythonhosted.org/packages/eb/a0/19119d8b7c05be49baf6c593f11c432d571b70d805f2fe94c0585e55e4c8/python_utils-2.3.0-py2.py3-none-any.whl
     |████████████████████████████████| 1.4MB 39.2MB/s eta 0:00:01
  Created wheel for smart-open: filename=smart_open-1.9.0-cp37-none-any.whl size=73088 sha256=089289bf274ceecefeef575b030d860ae0b6e36215b3f338b9436cc535601716
  Stored in directory: /home/ubuntu/.cache/pip/wheels/ab/10/93/5cff86f5b721d77edaecc

In [50]:
import chakin
chakin.search(lang='Japanese')

                         Name  Dimension     Corpus VocabularySize  \
6                fastText(ja)        300  Wikipedia           580K   
22  word2vec.Wiki-NEologd.50d         50  Wikipedia           335K   

                Method  Language                 Author  
6             fastText  Japanese               Facebook  
22  word2vec + NEologd  Japanese  Shiroyagi Corporation  


In [51]:
chakin.download(number=6, save_dir='.')

Test: 100% ||                                      | Time:  0:00:53  22.7 MiB/s


'./cc.ja.300.vec.gz'

私の環境では10分半ほどかかりました。

In [52]:
%%time
from gensim.models import KeyedVectors
wv = KeyedVectors.load_word2vec_format('cc.ja.300.vec.gz', binary=False)

CPU times: user 10min 30s, sys: 3.46 s, total: 10min 34s
Wall time: 10min 34s


In [53]:
nlp.vocab.reset_vectors(width=wv.vectors.shape[1])
for word in wv.vocab.keys():
    nlp.vocab[word]
    nlp.vocab.set_vector(word, wv[word])

In [55]:
nlp.vocab.vectors.name = 'fastText'

In [54]:
nlp.vocab.vectors.shape

(2113837, 300)

In [56]:
import cupy
with cupy.cuda.Device(0):
    nlp.vocab.vectors.data = cupy.asarray(nlp.vocab.vectors.data)

In [25]:
import random
from tqdm.notebook import tqdm
from spacy.util import minibatch, compounding

other_pipes = [pipe for pipe in nlp.pipe_names if pipe != "textcat"]
n_iter = 20

with nlp.disable_pipes(*other_pipes):  # only train textcat
    textcat = nlp.pipeline[-1][-1]
    #optimizer = textcat.begin_training(pretrained_vectors='fastText') # NOTE
    optimizer = textcat.begin_training() # NOTE
    print("Training the model...")
    batch_sizes = compounding(4.0, 32.0, 1.001)
    num_samples = len(train_data)
    for i in range(n_iter):
        print('===== iteration {}/{} ====='.format(i, n_iter))
        losses = {}
        # batch up the examples using spaCy's minibatch
        random.shuffle(train_data)
        batches = minibatch(train_data, size=batch_sizes)  # generator
        processed = 0
        for i, batch in tqdm(enumerate(batches)):
            texts, annotations = zip(*batch)
            nlp.update(texts, annotations, sgd=optimizer, drop=0.2, losses=losses)
            processed += len(batch)
            percentage = processed / num_samples * 100.0
            #if i % 20 == 0:
            #  print("  %5.2f %% of epoch done. batch size = %d" % (percentage, len(batch)))
        with textcat.model.use_params(optimizer.averages):
            # evaluate on the dev data split off in load_data()
            #scores = evaluate(nlp.tokenizer, textcat, val_docs, val_cats, verbose=True)
            report = evaluate(nlp.tokenizer, textcat, val_docs, val_cats, verbose=True)
        #print("{:^5}\t{:^5}\t{:^5}\t{:^5}".format("LOSS", "P", "R", "F"))
            print(report)
        #print("{:^5}\t{:^5}\t{:^5}\t{:^5}".format("LOSS", "P", "R", "F"))
        #print(
        #    "{0:.3f}\t{1:.3f}\t{2:.3f}\t{3:.3f}".format(  # print a simple table
        #        losses["textcat"],
        #        scores["textcat_p"],
        #        scores["textcat_r"],
        #        scores["textcat_f"],
        #    )
        #)

Training the model...
===== iteration 0/20 =====



                precision    recall  f1-score   support

dokujo-tsushin       0.73      0.81      0.77       790
  it-life-hack       0.73      0.78      0.75       816
 kaden-channel       0.68      0.41      0.51       386
livedoor-homme       0.68      0.50      0.58       545
   movie-enter       0.83      0.88      0.85       629
        peachy       0.70      0.74      0.72       800
          smax       0.86      0.88      0.87       961
  sports-watch       0.84      0.87      0.85       366
    topic-news       0.78      0.81      0.79       370

      accuracy                           0.76      5663
     macro avg       0.76      0.74      0.74      5663
  weighted avg       0.76      0.76      0.76      5663

===== iteration 1/20 =====



                precision    recall  f1-score   support

dokujo-tsushin       0.76      0.82      0.79       790
  it-life-hack       0.78      0.81      0.80       816
 kaden-channel       0.71      0.52      0.60       386
livedoor-homme       0.72      0.58      0.64       545
   movie-enter       0.83      0.89      0.86       629
        peachy       0.75      0.76      0.75       800
          smax       0.87      0.90      0.89       961
  sports-watch       0.84      0.88      0.86       366
    topic-news       0.80      0.82      0.81       370

      accuracy                           0.79      5663
     macro avg       0.79      0.78      0.78      5663
  weighted avg       0.79      0.79      0.79      5663

===== iteration 2/20 =====



                precision    recall  f1-score   support

dokujo-tsushin       0.78      0.82      0.80       790
  it-life-hack       0.79      0.83      0.81       816
 kaden-channel       0.73      0.59      0.66       386
livedoor-homme       0.75      0.64      0.69       545
   movie-enter       0.85      0.89      0.87       629
        peachy       0.77      0.78      0.77       800
          smax       0.88      0.91      0.89       961
  sports-watch       0.85      0.89      0.87       366
    topic-news       0.81      0.81      0.81       370

      accuracy                           0.81      5663
     macro avg       0.80      0.79      0.80      5663
  weighted avg       0.81      0.81      0.81      5663

===== iteration 3/20 =====



                precision    recall  f1-score   support

dokujo-tsushin       0.79      0.82      0.81       790
  it-life-hack       0.80      0.83      0.82       816
 kaden-channel       0.72      0.62      0.67       386
livedoor-homme       0.76      0.65      0.70       545
   movie-enter       0.86      0.90      0.88       629
        peachy       0.77      0.79      0.78       800
          smax       0.89      0.91      0.90       961
  sports-watch       0.86      0.89      0.87       366
    topic-news       0.84      0.83      0.83       370

      accuracy                           0.82      5663
     macro avg       0.81      0.80      0.81      5663
  weighted avg       0.81      0.82      0.81      5663

===== iteration 4/20 =====


...


                precision    recall  f1-score   support

dokujo-tsushin       0.83      0.84      0.83       790
  it-life-hack       0.82      0.84      0.83       816
 kaden-channel       0.76      0.67      0.71       386
livedoor-homme       0.77      0.71      0.74       545
   movie-enter       0.87      0.91      0.89       629
        peachy       0.82      0.82      0.82       800
          smax       0.89      0.92      0.91       961
  sports-watch       0.89      0.91      0.90       366
    topic-news       0.84      0.84      0.84       370

      accuracy                           0.84      5663
     macro avg       0.83      0.83      0.83      5663
  weighted avg       0.84      0.84      0.84      5663

===== iteration 16/20 =====



                precision    recall  f1-score   support

dokujo-tsushin       0.83      0.84      0.84       790
  it-life-hack       0.83      0.85      0.84       816
 kaden-channel       0.76      0.66      0.71       386
livedoor-homme       0.77      0.72      0.74       545
   movie-enter       0.87      0.90      0.89       629
        peachy       0.82      0.81      0.82       800
          smax       0.89      0.92      0.91       961
  sports-watch       0.90      0.90      0.90       366
    topic-news       0.83      0.85      0.84       370

      accuracy                           0.84      5663
     macro avg       0.83      0.83      0.83      5663
  weighted avg       0.84      0.84      0.84      5663

===== iteration 17/20 =====



                precision    recall  f1-score   support

dokujo-tsushin       0.84      0.84      0.84       790
  it-life-hack       0.83      0.86      0.84       816
 kaden-channel       0.77      0.67      0.71       386
livedoor-homme       0.78      0.73      0.75       545
   movie-enter       0.87      0.90      0.89       629
        peachy       0.83      0.81      0.82       800
          smax       0.89      0.92      0.91       961
  sports-watch       0.90      0.90      0.90       366
    topic-news       0.84      0.85      0.85       370

      accuracy                           0.84      5663
     macro avg       0.84      0.83      0.83      5663
  weighted avg       0.84      0.84      0.84      5663

===== iteration 18/20 =====



                precision    recall  f1-score   support

dokujo-tsushin       0.84      0.84      0.84       790
  it-life-hack       0.83      0.85      0.84       816
 kaden-channel       0.76      0.66      0.71       386
livedoor-homme       0.78      0.72      0.75       545
   movie-enter       0.87      0.91      0.89       629
        peachy       0.81      0.81      0.81       800
          smax       0.89      0.92      0.91       961
  sports-watch       0.89      0.90      0.90       366
    topic-news       0.84      0.85      0.84       370

      accuracy                           0.84      5663
     macro avg       0.83      0.83      0.83      5663
  weighted avg       0.84      0.84      0.84      5663

===== iteration 19/20 =====



                precision    recall  f1-score   support

dokujo-tsushin       0.83      0.84      0.84       790
  it-life-hack       0.82      0.84      0.83       816
 kaden-channel       0.75      0.66      0.70       386
livedoor-homme       0.78      0.71      0.75       545
   movie-enter       0.87      0.91      0.89       629
        peachy       0.81      0.81      0.81       800
          smax       0.89      0.92      0.91       961
  sports-watch       0.90      0.90      0.90       366
    topic-news       0.83      0.85      0.84       370

      accuracy                           0.84      5663
     macro avg       0.83      0.83      0.83      5663
  weighted avg       0.84      0.84      0.84      5663



#### BERT

In [ ]:
import torch
from transformers import BertForSequenceClassification
net = BertForSequenceClassification.from_pretrained('bert-base-japanese-whole-word-masking', num_labels=9)
device='cuda' if torch.cuda.is_available() else 'cpu'
net.to(device)

このニューラルネットワークの構造をnetron (https://github.com/lutzroeder/netron) というツールを用いて可視化すると次のようになります。
<img src="../figures/bert_classifier_netron.png" alt="bert_classifier_netron" width="150">
BERTモデルの構造が `BertModel` に押し込められているため、やけにシンプルに見えますが、ここではあまり深く考えないようにします。

PyTorchを用いてディープラーニングを実装する際には

In [ ]:
from torchtext.data import Field
from transformers import BertJapaneseTokenizer
tokenizer = BertJapaneseTokenizer.from_pretrained('bert-base-japanese-whole-word-masking')

import re
import mojimoji

def tokenizer_with_preprocessing(text):
        # 半角、全角の変換
        text = mojimoji.han_to_zen(text)
        # 改行、半角スペース、全角スペースを削除
        text = re.sub('\r', '', text)
        text = re.sub('\n', '', text)
        text = re.sub('　', '', text)
        text = re.sub(' ', '', text)
        # 数字文字の一律「0」化
        text = re.sub(r'[0-9 ０-９]', '0', text)  # 数字
        return tokenizer.tokenize(text)
    
TEXT = Field(
    sequential=True,  
    tokenize=tokenizer_with_preprocessing, 
    use_vocab=True,
    lower=False,
    include_lengths=True,
    batch_first=True,
    fix_length=512,
    init_token='[CLS]',
    eos_token='[SEP]',
    pad_token='[PAD]',
    unk_token='[UNK]'
)
LABEL = Field(sequential=False, use_vocab=False)

In [ ]:
from torchtext.data import TabularDataset
train, val, test = TabularDataset.splits(
    path='.', train='train.tsv', validation='val.tsv', test='test.tsv', format='tsv', 
    fields=[('body', TEXT), ('service', LABEL)], skip_header=True)

In [ ]:
TEXT.build_vocab(train, min_freq=1)
TEXT.vocab.stoi = tokenizer.vocab

In [ ]:
import torch
from torchtext.data import Iterator
batch_size = 32
#train_iter, val_iter, test_iter = Iterator.splits((train, val, test), batch_size=batch_size, device='cuda' if torch.cuda.is_available() else 'cpu')
train_iter = Iterator(train, batch_size, train=True, device=device)
val_iter = Iterator(val, batch_size, train=False, sort=False, device=device)
test_iter = Iterator(test, batch_size, train=False, sort=False, device=device)

In [ ]:
iterator_dict = {'train': train_iter, 'val': val_iter, 'test': test_iter}

In [ ]:
"""
net.to('cuda')
batch = next(iter(train_iter))
inputs = batch.body[0]
labels = batch.service
loss, logit = net(inputs, labels=labels)
"""

In [ ]:
for name, param in net.named_parameters():
    param.requires_grad = False

In [ ]:
for name, param in net.bert.encoder.layer[-1].named_parameters():
    param.requires_grad = True

In [ ]:
for name, param in net.classifier.named_parameters():
    param.requires_grad = True

In [ ]:
optimizer = torch.optim.Adam([
    {'params': net.bert.encoder.layer[-1].parameters(), 'lr': 5e-5},
    {'params': net.classifier.parameters(), 'lr': 5e-5}
], betas=(0.9, 0.999))

In [ ]:
criterion = torch.nn.CrossEntropyLoss()

In [ ]:
from datetime import datetime
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter(log_dir='./logs/' + datetime.today().isoformat(timespec='seconds'))

def train_model(net, iterator_dict, criterion, optimizer, num_epochs):
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    
    iteration = 1
    for epoch in range(num_epochs):
        for phase in ['train', 'val']:
            if phase == 'train':
                net.train()
            else:
                net.eval()
                
            epoch_loss = 0.
            epoch_corrects = 0
            
            for batch in iterator_dict[phase]:
                inputs = batch.body[0]
                labels = batch.service
                optimizer.zero_grad()
                with torch.set_grad_enabled(phase == 'train'):
                    loss, logit = net(input_ids=inputs, labels=labels)
                    #print(loss, logit)
                    _, preds = torch.max(logit, 1)
                    #predictions.append(preds.cpu().numpy())
                    #ground_truths.append(labels.data.cpu().numpy())
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                        
                        if (iteration % 10 == 0):
                            acc = (torch.sum(preds == labels.data)).double() / batch_size
                            print('iteration {} || Loss: {:.4f} || acc {}'.format(
                                iteration, loss.item(), acc.item()))
                            writer.add_scalar("Loss vs Iteration/{}".format(phase), loss.item(), iteration)
                            writer.add_scalar("Accuracy vs Iteration/{}".format(phase), acc.item(), iteration)
                        iteration += 1
                    
                    epoch_loss += loss.item() * batch_size
                    #print(preds, labels.data)
                    epoch_corrects += torch.sum(preds == labels.data)
        epoch_loss = epoch_loss / len(iterator_dict[phase].dataset)
        epoch_acc = epoch_corrects.double() / len(iterator_dict[phase].dataset)
        writer.add_scalar("Loss vs Epoch/{}".format(phase), epoch_loss, epoch + 1)
        writer.add_scalar("Accuracy vs Epoch/{}".format(phase), epoch_acc, epoch + 1)
        
        print('Epoch {}/{} | {} | Loss: {:.4f} Acc: {:.4f}'.format(
            epoch + 1, num_epochs, phase, epoch_loss, epoch_acc))
    return net

In [ ]:
num_epochs = 100 
net_trained = train_model(net, iterator_dict, criterion, optimizer, num_epochs)

Early stopping追加

### 評価と比較

まずBoWからはじめよう

## まとめ

BoW、CNN、BERT

難しいタスクならBERT試すのもありかも

## 参考文献

- [(Part 1) tensorflow2でhuggingfaceのtransformersを使ってBERTを文書分類モデルに転移学習する](https://tksmml.hatenablog.com/entry/2019/10/22/215000)
- [(Part 2) tensorflow 2 でhugging faceのtransformers公式のBERT日本語学習済みモデルを文書分類モデルにfine-tuningする](https://tksmml.hatenablog.com/entry/2019/12/15/090900)
- [All Models and checkpoints](https://huggingface.co/models)
- [Working with GPU packages](https://docs.anaconda.com/anaconda/user-guide/tasks/gpu-packages/)
- [gensimとPyTorchを使ったlive doorニュースコーパスのテキスト分類](https://www.pytry3g.com/entry/2018/04/03/194202)
- [bert-japanese](https://github.com/cl-tohoku/bert-japanese)
- [DocumentClassificationUsingBERT-Japanese](https://github.com/nekoumei/DocumentClassificationUsingBERT-Japanese)
- [torchtext](https://torchtext.readthedocs.io/en/latest/index.html)
- [FX予測 : PyTorchのBERTで経済ニュース解析](https://qiita.com/THERE2/items/8b7c94787911fad8daa6)
- [torchtextで簡単にDeepな自然言語処理](https://qiita.com/itok_msi/items/1f3746f7e89a19dafac5)
- [transformers](https://github.com/huggingface/transformers)
- [BERTを使った文章要約 [身内向け]](https://qiita.com/IwasakiYuuki/items/25f5bbcde4f82dff7f1a)
- [MeCab + Gensim による日本語の自然言語処理](https://www.koi.mashykom.com/nlp.html)
- [論文解説 Google's Neural Machine Translation System: Bridging the Gap between Human and Machine Translation (GNMT)](http://deeplearning.hatenablog.com/entry/gnmt)
- [BERT with SentencePiece で日本語専用の pre-trained モデルを学習し、それを基にタスクを解く](https://techlife.cookpad.com/entry/2018/12/04/093000)
- [はじめての自然言語処理](https://www.ogis-ri.co.jp/otc/hiroba/technical/similar-document-search/)